In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing necessary libraries and David's reader module

In [ ]:
import sys
from tqdm import tqdm
sys.path.append('/content/drive/MyDrive/AUT 2023/DATA 512/P1/wildfire')

In [ ]:
#
#    IMPORTS
#

#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of California wildfires extracted from the main wildfire dataset.
#

from Reader import Reader as WFReader


In [ ]:
SAMPLE_DATA_FILENAME = '/content/drive/MyDrive/AUT 2023/DATA 512/P1/USGS_Wildland_Fire_Combined_Dataset.json'

In [ ]:
#
#    This bit of code opens a new wildfire reader, gets the header information and prints it to the screen
#
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(SAMPLE_DATA_FILENAME)
print()
#
#    Now print the header - it contains some useful information
#
header_dict = wfreader.header()
header_keys = list(header_dict.keys())
print("The header has the following keys:")
print(header_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))

Attempting to open '/content/drive/MyDrive/AUT 2023/DATA 512/P1/USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object

The header has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields']

Header Dictionary
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire IRWIN IDs",
        "Listed_Fire_Dates":

Load all wildfire occurrences i.e. features

In [ ]:
#
#    This sample code will load the whole sample file, or a small amount of the complete dataset.
#
#MAX_FEATURE_LOAD = 100
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 10000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    # if feature_count >= MAX_FEATURE_LOAD:
    #     break
    feature = wfreader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")




Loaded 10000 features
Loaded 20000 features
Loaded 30000 features
Loaded 40000 features
Loaded 50000 features
Loaded 60000 features
Loaded 70000 features
Loaded 80000 features
Loaded 90000 features
Loaded 100000 features
Loaded 110000 features
Loaded 120000 features
Loaded 130000 features
Loaded a total of 135061 features
Variable 'feature_list' contains 135061 features


In [ ]:
#
#    Every feature has a 'geometry' which specifies geo coordinates that make up each geographic thing
#    In the case of the wildfire data, most wildfires are bounded shapes, circles, squares, etc. This is
#    represented by shapes called 'rings' in GeoJSON.
#
# Get the geometry for the feature we pulled from the feature_list
wf_geometry = wf_feature['geometry']
# The largest shape (ring) is supposed to be item zero in the list of 'rings'
wf_bigest_ring = wf_geometry['rings'][0]

print(f"The largest ring of wf_feature['features'][{SLOT}]['rings'] consists of {len(wf_bigest_ring)} points.")

The largest ring of wf_feature['features'][0]['rings'] consists of 768 points.


In [ ]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [ ]:
#
#   Convert one ring from the default to EPSG
#
#   There are two options here - depending upon whether you loaded data useing GeoJSON or the wildfire.Reader
#
#ring_in_epsg4326 = convert_ring_to_epsg4326(gj_bigest_ring)
#
ring_in_epsg4326 = convert_ring_to_epsg4326(wf_bigest_ring)
#
print(f"Ring consists of {len(ring_in_epsg4326)} points.")
#
#    If you want to print them out you can see what they look like converted.
#print(ring_in_epsg4326)
#for point in ring_in_epsg4326:
#    print(f"{point[0]},{point[1]}")

Ring consists of 768 points.


Add in location details corresponding to the city allotted to me, 'North Platte, Nebraska'

In [ ]:
CITY_LOCATIONS = {
    'np' :     {'city'   : 'North Platte',
                       'latlon' : [41.1403, -100.7601] }
}


I am considering the average distance of a fire feature polygon to the specified place. I think it will be a better estimator than shortest distance.

In [ ]:
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average

#FILTERING PROCESS:

Including only those fire locations which are within 1250 miles of North Platte. Also including fires which happened after 1963.

#SMOKE ESTIMATE CALCULATION:

In the given dataset, the parameters which give quantitative estimates of the fire are:



1.   The Area Burnt by the Fire (direct proportion)
2.   The Distance of the Fire from North Platte (inverse proportion)
3.   The Intensity of the Fire (direct proportion)

The fire intensity can be computed based on the type of the fire (Wildfire, Prescribed Fire etc.) The metadata of the geographic data reveals that wildfire has highest intensity since they are proper fires. Prescribed fires might be smaller fires, hence I assigned a multiplying factor the same and calculated as follows:

`Fire-Estimate = (Area*Intensity)/(Distance)`



In [ ]:
fire_intensity = dict()
fire_intensity['Wildfire'] = 2
fire_intensity['Likely Wildfire'] = 1.75
fire_intensity['Unknown - Likely Wildfire'] = 1.5
fire_intensity['Prescribed Fire'] = 1.25
fire_intensity['Unknown - Likely Prescribed Fire'] = 1

In [ ]:
place = CITY_LOCATIONS['np']
rel_fires = []
for wf_feature in tqdm(feature_list):
    #print(f"{place['city']}")
    try:
      wf_year = wf_feature['attributes']['Fire_Year']
      wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
      wf_size = wf_feature['attributes']['GIS_Acres']
      wf_type = wf_feature['attributes']['Assigned_Fire_Type']
      ring_data = wf_feature['geometry']['rings'][0]
    except:
      print("Exception occurred, ring param was missing")
        #     Compute using the average distance to all points on the perimeter
    if wf_year >= 1963:
      distance = average_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
      if distance < 1250:
        ring = convert_ring_to_epsg4326(ring_data)
        wf_feature['attributes']['distance'] = distance
        smoke_estimate = (wf_size*fire_intensity[wf_type])/distance
        wf_feature['attributes']['smoke_estimate'] = smoke_estimate
        rel_fires.append(wf_feature['attributes'])

 81%|████████  | 109605/135061 [3:50:35<57:23,  7.39it/s]

Exception occurred, ring param was missing


 82%|████████▏ | 110224/135061 [3:52:10<1:00:27,  6.85it/s]

Exception occurred, ring param was missing


 82%|████████▏ | 110639/135061 [3:53:14<1:03:26,  6.42it/s]

...

 93%|█████████▎| 125745/135061 [4:31:09<22:16,  6.97it/s]

Exception occurred, ring param was missing


 94%|█████████▍| 127492/135061 [4:35:31<18:06,  6.97it/s]

Exception occurred, ring param was missing


100%|██████████| 135061/135061 [4:54:45<00:00,  7.64it/s]


In [ ]:
len(rel_fires)

106436

I have around 100000 fires which fall within the specified constraints for North Platte. I would attribute this high number to the fact that Nebraska is in the middle of the country and a radius of 1250 miles would encompass states in all directions.

I save this data in a JSON file.

In [ ]:
int_json = open('/content/drive/MyDrive/AUT 2023/DATA 512/P1/smoke_estimate_NP.json', "w")
json.dump(rel_fires, int_json, indent = 4)
int_json.close()

In [ ]:
import pandas as pd
df = pd.read_json('/content/drive/MyDrive/AUT 2023/DATA 512/P1/smoke_estimate_NP.json')
df.head()

,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Prescribed_Burn_Notice,Wildfire_and_Rx_Flag,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,distance,smoke_estimate
0,14299,14299,Wildfire,1963,1,"1 (1), 3 (3)",40992.458271,16589.059302,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,Prescribed fire data in this dataset represent...,None,None,0.385355,NaN,No,73550.428118,1.658906e+08,834.453121,98.249877
1,14300,14300,Wildfire,1963,1,"1 (1), 3 (3)",25757.090203,10423.524591,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (2)",...,Prescribed fire data in this dataset represent...,None,None,0.364815,NaN,No,59920.576713,1.042352e+08,860.579136,59.859899
2,14301,14301,Wildfire,1963,1,"1 (5), 3 (15), 5 (1)",45527.210986,18424.208617,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (6), Likely Wildfire (15)",...,Prescribed fire data in this dataset represent...,None,None,0.320927,NaN,No,84936.827810,1.842421e+08,826.551127,110.161875
3,14302,14302,Wildfire,1963,1,"1 (1), 3 (3), 5 (1)",10395.010334,4206.711433,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (3)",...,Prescribed fire data in this dataset represent...,None,None,0.428936,NaN,No,35105.903602,4.206711e+07,782.078523,26.583035
4,14303,14303,Wildfire,1963,1,"1 (1), 3 (3)",9983.605738,4040.221900,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,Prescribed fire data in this dataset represent...,None,None,0.703178,NaN,No,26870.456126,4.040222e+07,820.831455,24.325592
